In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Set Paths
# Replace 'path_to_zip_in_drive' with the actual path to your zip file in Google Drive
zip_path = '/content/drive/MyDrive/PADDY IMAGES.zip'  # Example: 'dataset.zip'
extract_to = '/content/'  # Destination folder in Colab

# Step 3: Extract the Zip File
import zipfile
import os

# Create the extraction folder if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Extraction complete!")
print(f"Files extracted to: {extract_to}")

# Optional: List extracted files
print("Extracted files:")
print(os.listdir(extract_to))


Mounted at /content/drive
Extraction complete!
Files extracted to: /content/
Extracted files:
['.config', 'PADDY IMAGES', 'drive', 'sample_data']


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Define input and output directories
input_dir = "/content/PADDY IMAGES"  # Path to your dataset
output_dir = "/content/preprocessed"  # Output folder for preprocessed images

# Preprocessing parameters
image_size = (299, 299)  # Input size for Xception model

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to preprocess and save images
def preprocess_and_save(input_path, output_path):
    try:
        # Read the image
        image = cv2.imread(input_path)
        if image is None:
            print(f"Error reading image: {input_path}")
            return

        # Resize the image
        resized_image = cv2.resize(image, image_size)

        # Normalize the image (scale pixel values to [0, 1])
        normalized_image = resized_image / 255.0

        # Convert back to uint8 for saving (scale [0, 1] to [0, 255])
        output_image = (normalized_image * 255).astype(np.uint8)

        # Save the preprocessed image
        cv2.imwrite(output_path, output_image)
    except Exception as e:
        print(f"Error processing image {input_path}: {e}")

# Iterate through the input dataset
for folder_name in os.listdir(input_dir):
    folder_path = os.path.join(input_dir, folder_name)

    # Ensure it's a directory
    if os.path.isdir(folder_path):
        # Create the corresponding output folder
        output_folder_path = os.path.join(output_dir, folder_name)
        os.makedirs(output_folder_path, exist_ok=True)

        # Process each image in the folder
        for image_name in tqdm(os.listdir(folder_path), desc=f"Processing {folder_name}"):
            input_image_path = os.path.join(folder_path, image_name)
            output_image_path = os.path.join(output_folder_path, image_name)

            # Preprocess and save the image
            preprocess_and_save(input_image_path, output_image_path)

print("Preprocessing complete. Preprocessed images are saved in the 'preprocessed' folder.")


Processing bacterial_leaf_streak: 100%|██████████| 380/380 [00:04<00:00, 82.04it/s]

Preprocessing complete. Preprocessed images are saved in the 'preprocessed' folder.


In [ ]:
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2  # Using a lighter model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Paths
data_dir = "/content/preprocessed"  # Path to the preprocessed dataset
input_shape = (128, 128, 3)  # Reduced input shape for faster processing

# Data Generators for Train and Validation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=16,  # Smaller batch size for faster iterations
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=16,
    class_mode="categorical",
    subset="validation",
    shuffle=True
)

# Class names (folders)
class_names = list(train_generator.class_indices.keys())
print("Class Names:", class_names)

# Build the MobileNetV2 Model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(class_names), activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers (transfer learning)
for layer in base_model.layers:
    layer.trainable = False

# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Train the Model
epochs = 50  # Increased epochs for more thorough training
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

# Randomly Select 10 Images for Testing
test_images = []
test_labels = []

for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    images = os.listdir(folder_path)
    selected_images = random.sample(images, min(10, len(images)))  # Select 10 or fewer if not enough
    for img in selected_images:
        img_path = os.path.join(folder_path, img)
        test_images.append(img_path)
        test_labels.append(folder)

# Predict on the Test Images and Calculate Accuracy
correct_predictions = 0

print("\nTesting on Randomly Selected Images:")
for img_path, true_label in zip(test_images, test_labels):
    img = load_img(img_path, target_size=input_shape[:2])
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array, verbose=0)
    predicted_label = class_names[np.argmax(prediction)]

    is_correct = true_label == predicted_label
    if is_correct:
        correct_predictions += 1

    print(f"Image: {img_path}")
    print(f"True Label: {true_label}, Predicted Label: {predicted_label} {'[CORRECT]' if is_correct else '[WRONG]'}\n")

# Calculate and Display Accuracy
total_test_images = len(test_images)
accuracy = (correct_predictions / total_test_images) * 100
print(f"Testing Accuracy: {accuracy:.2f}% ({correct_predictions}/{total_test_images} correct predictions)")


Found 8330 images belonging to 10 classes.
Found 2077 images belonging to 10 classes.
Class Names: ['bacterial_leaf_blight', 'bacterial_leaf_streak', 'bacterial_panicle_blight', 'blast', 'brown_spot', 'dead_heart', 'downy_mildew', 'hispa', 'normal', 'tungro']
Epoch 1/50
521/521 ━━━━━━━━━━━━━━━━━━━━ 151s 279ms/step - accuracy: 0.3782 - loss: 1.7882 - val_accuracy: 0.6172 - val_loss: 1.1909
Epoch 2/50
521/521 ━━━━━━━━━━━━━━━━━━━━ 144s 277ms/step - accuracy: 0.6508 - loss: 1.0803 - val_accuracy: 0.6688 - val_loss: 1.0237
Epoch 3/50
521/521 ━━━━━━━━━━━━━━━━━━━━ 201s 274ms/step - accuracy: 0.7226 - loss: 0.8718 - val_accuracy: 0.6952 - val_loss: 0.9455
Epoch 4/50
521/521 ━━━━━━━━━━━━━━━━━━━━ 217s 303ms/step - accuracy: 0.7570 - loss: 0.7919 - val_accuracy: 0.7256 - val_loss: 0.8571
Epoch 5/50
521/521 ━━━━━━━━━━━━━━━━━━━━ 187s 274ms/step - accuracy: 0.7885 - loss: 0.7031 - val_accuracy: 0.7472 - val_loss: 0.8091
Epoch 6/50
521/521 ━━━━━━━━━━━━━━━━━━━━ 143s 275ms/step - accuracy: 0.8042 - los